In [1]:
import os
import sys
import numpy as np
import json
import matplotlib.pyplot as plt
import MDAnalysis
import urllib.request
import yaml
import random


# This defines the path for the NMRlipids databank on your computer. 
# Default is that this repository and the NMRlipids databank repository are cloned to the same folder.
# If this is not the case, change this to the folder where the NMRlipids databank repository is located.
databankPath =  '../../Databank/'

# This enables the access to functions defined in the NMRlipids databank.
sys.path.insert(1, databankPath + '/Scripts/BuildDatabank/')
from databankLibrary import * 

# This initializes the databank and stores the information of all simulations into a list.
# Each list item contains the information from README.yaml file of the given simulation.
systems = initialize_databank(databankPath)


In [67]:
### Put your composition with NMRlipids databank universal molecule names into the 'InterestingCompositions' array,
### and molecular ratios to the 'MolecularRatios' array. The code will then predict the are per lipid for your 
### composition using linear regression model based on the NMRlipids databank.

InterestingComposition = ['POPC','POPE','POPS', 'CHOL', 'POPI']
MolecularRatios = [0.68, 0.21,0.04,0.07,0.0]

#InterestingComposition = ['POPS', 'POPE','POPC']

#InterestingComposition = ['POPC','CHOL']
#MolecularRatios = [1, 0]

aplArray = []
interestArray = []

for system in systems:
    skip = True
    
    for interest in InterestingComposition:
        if interest in system['COMPOSITION']:
            skip = False
    
    if skip:
        continue

    #if 'CHARMM' not in system['FF']:
    #    continue
    
    #if system['TEMPERATURE'] < 304:
    #    continue
    
        
    interestArrayTMP = []
    for interest in InterestingComposition:
        try:
            interestArrayTMP.append(sum(system['COMPOSITION'][interest]['COUNT']) / GetNlipids(system))
        except:
            interestArrayTMP.append(0)
    
    try:
        aplArray.append(CalcAreaPerMolecule(system))
        interestArray.append(interestArrayTMP)
    except:
        print('Skipped ' + system['path'])
        continue
    
    #print(system['COMPOSITION'])
    #print(CalcAreaPerMolecule(system))
    #print()
    
    
    
from sklearn.linear_model import LinearRegression

X = np.array(interestArray)
y = np.array(aplArray)

# Create an instance of the LinearRegression class
reg = LinearRegression()
 
# Fit the model to the data
reg.fit(X, y)
 
# Print the coefficients of the model
print('Linear regression model coefficents: ' , reg.coef_, '\n')


predictedAPL = reg.predict([MolecularRatios])
print('Predicted area per lipid for membrane with ' , 
      InterestingComposition, 'with the molecular ratios of ' , MolecularRatios, ':\n')
print(predictedAPL[0], 'Å^2')

Linear regression model coefficents:  [ -1.14095308  -5.29230581  -7.24782914 -45.20973488 -53.597217  ] 

Predicted area per lipid for membrane with  ['POPC', 'POPE', 'POPS', 'CHOL', 'POPI'] with the molecular ratios of  [0.68, 0.21, 0.04, 0.07, 0.0] :

58.63840873180086 Å^2


In [11]:
CompDict = {'POPC': 864,
'POPE': 336,
'POPS': 64,
'CHL1': 112,
'POPI': 160,
'PSM': 64}

Nlipids = 0
for i in CompDict:
    Nlipids += CompDict[i]
print(Nlipids)

for i in CompDict:
    print(i, CompDict[i]/Nlipids)

1600
POPC 0.54
POPE 0.21
POPS 0.04
CHL1 0.07
POPI 0.1
PSM 0.04
